# Perceptual Losses for Real-Time Style Transfer and Super-Resolution
(실시간 style transfer와 초고해상도를 위한 지각 손실 함수)

# Abstract
- 우리는 입력 이미지가 출력 이미지로 변환되는 이미지 변환 문제(image transformation problem)를 고려한다.
- 이러한 문제에 대한 최신의 방법은 일반적으로 feed-forward convolutional neural network를 훈련시킨다. 출력 이미지와 지상 실측(ground-truth) 이미지 사이의 픽셀당 손실(per-pixel loss)를 사용해서.
- 병렬작업(parallel work)은 고품질의 이미지를 생성가능하다. 사전에 훈련된(pretrained) 네트워크로부터 추출된 고수준 feature를 기반으로 지각 손실함수(perceptual loss function)을 정의하고 최적화함으로써.
- 우리는 두 접근방식(approach)의 장점을 결합한다. 그리고, 이미지 변환 작업을 위한 feed-forward network를 훈련하기 위한 지각 손실함수(perceptual loss function)을 제안한다.
- 우리는 image style transfer의 결과를 보여준다. 실시간으로 Gatys 등등이 제안한 최적화 방법을 해결하기 위해 feed-forward network는 훈련된다.
- 최적화 기반 방법(optimization-based method)과 비교하여, 우리의 네트워크는 비슷한 질적인(qualitative) 결과를 준다. 하지만, 3배 규모(magnitude)로 더 빠르다.
- 우리는 또한 단일 이미지(single-image) 초고해상도(super-resolution)을 실험한다. 픽셀당 손실(per-pixel loss)을 지각 손실(perceptual loss)로 대체하는 것은 시각적으로 만족스러운(pleasing) 결과를 준다.

# 1. Introduction
- 많은 고전적인(classic) 문제는 이미지 변한 작업으로써 구성될(frame) 수 있다. 시스템은 몇 개의 입력 이미지를 받고 출력 이미지로 변환한다.
- 이미지 처리(image processing)
  - 예) 노이즈 제거(denoising), 초고해상도(super-resolution), 채색(colorization)
  - 입력 : 저하된 이미지(degraded image)
    - 예) 노이즈(noisy), 저해상도(resolution), 회색조(grayscale)
  - 출력 : 고품질 컬러 이미지
- 컴퓨터 비전(computer vision)
  - 예) 의미론적 분할(semantic segmentation), 깊이 추정(depth estimation)
  - 입력 : 컬러 이미지
  - 출력 : 장면(scene)에 대한 의미론적인 또는 기하학적인 정보를 인코딩한다.
- 이미지 변환 작업(image transformation task)을 해결하기 위한 1가지 접근방식은 지도학습 방법(supervised manner)로 feedforward convolutional neural network를 훈련시키는 것이다. 픽셀당 손실 함수를 사용하여 출력 이미지와 실측 이미지 간의 차이를 측정한다.
- 이러한 접근 방식은 초해상도(super-resolution)을 위해 Dong 등등에 의해 사용되었다. 채색(colorization)을 위해 Cheng 등등에 의해 사용되었다. 분할(segmentation)을 위해 Long 등등에 의해 사용되었다. 깊이(depth) 및 표면 정규 예측(법선 벡터 예측, surface normal prediction)을 위해 Eigen 등등에 의해 사용되었다.
- 이러한 접근 방식은 테스트 시간(test-time)에 효율적이다. 오직 훈련된(trained) 네트워크를 통한 전달(forward pass)만 필요하다.
- 하지만, 이러한 방법에서 사용되는 픽셀당 손실은 출력 이미지와 실측 이미지 사이의 지각적 차이(perceptual diference)를 포착하지 못한다.
- 예를 들어, 1개의 픽셀만큼 서로 상쇄된(offset) 2개의 동일한 이미지를 고려해보자. ; 드릉릐 지각적인 유사성에도 불구하고, 그들은 아마도 픽셀당 손실에 의해 측정될 때 매우 다를 것이다.
- 이와 동시에, 최근 연구에 따르면, 고품질의 이미지는 픽셀 간의 차이 대신에 지각 손실 함수를 사용하여 만들 수 있다고 한다. 그러나 대신에, 사전학습된 컨볼루젼 신경망에서 추출된 고수준 이미지 특징(feature) 표현(representation) 간의 차이에 대한 것이다.
- 이미지는 손실 함수를 최소화함으로써 생성된다.
- 이 전략(strategy)는 Mahendran 등등에 의한 특징 전도(feature inversion), Simonya, Yosinski 등등에 의한 특징 시각화(feature visualization), Gatys 등등에 의한 텍스쳐 합성(texture synthesis)과 스타일 전달(style transfer)에 적용되었다.
- 이러한 접근 방식은 고품질 이미지를 생성한다. 하지만, 느리다. 왜냐하면, 추론(inference)은 최적화 문제를 해결해야 하기 때문이다.
- 이 논문에서, 우리는 이러한 2가지 접근 방식의 장점을 결합했다.
- 우리는 이미지 변환 작업을 위해 feedforward 변환 네트워크를 훈련시켰다. 하지만, 낮은 수준의(low-level) 픽셀 정보에만 의존하는 픽셀당 손실 함수를 사용하는 대신, 우리는 사전훈련된 손실 네트워크(pretrained loss network)의 고수준 특징에 의존하는 지각 손실 함수(perceptual loss function)을 사용하여 네트워크를 훈련한다.
- 훈련 중에, 지각 손실은 픽셀당 손실보다 이미지 유사성(image similarity)을 더 강력하게(robustly) 측정한다. 그리고, 테스트 시간(test-time)에, 변환 네트워크가 실시간으로(real-time) 작동된다.
- 우리는 2가지 작업을 실험한다.: 스타일 전송(style transfer), 단일 이미지 초고해상도
- 두 작업 다 본질적으로(inherently) 타당하지 않다.(ill-posed); 스타일 전송의 경우 올바른 단일 출력이 없다. 그리고 초고해상도의 경우 동일한 저해상도 입력을 생성할 수 있는 고해상도 이미지가 있다.
- 두 작업에서 성공하려면 입력 이미지에 대한 의미론적 추론(semantic reasoning)이 필요하다.
- 스타일 전송의 경우, 출력은 입력과 의미상(semantically) 유사해야 한다. 색상, 질감의 급격한(drastic) 변화에도 불구하고.
- 초고해상도의 경우, 시각적으로 애매모호한 저해상도 입력에서 질 높은(fine) 세부사항(detail)을 추론(infer)해야한다.
- 원칙적으로, 각각의 작업에 훈련된 고용량(high-capacity) 신경망은 관련 의미론(relevant semantic)에 대해 추론(reason)하는 방법을 암시적으로(implicitly) 학습할 수 있다. 하지만, 실제는, 우리는 처음(스크래치, scratch)부터 학습할 필요가 없다.: 지각 손실 함수의 사용은 손실 네트워크(loss network)에서 변환 네트워크(transformation network)로 의미 지식(semantic knowledge)을 전달할 수 있다.
- 스타일 전송을 위해, 우리의 feed-forward network는 논문 "Gatys, L.A., Ecker, A.S., Bethge, M.: A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576 (2015)
"의 최적화 문제를 해결하도록 훈련되었다. ; 우리의 결과는 질적으로 그리고 목적 함수(=손실 함수, objective function) 값에 의해 측정된 것과 유사하다. 하지만, 생성하는데 걸리는 속도는 3배 빠르다.
- 초고해상도의 경우, 우리는 픽셀당 손실을 지각 손실로 대체하는 것이 x4, x8 초고해상도에서 시각적으로 만족스러운 결과를 준다는 것을 보여준다.
- <img src="https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig1_HTML.gif">
  
  - 그림 1.
  - 위 그림 : 스타일 전송(style transfer) 예시
  - 아래 그림 : x4 초고해상도(super-resolution) 예시
  - 스타일 전송(style transfer)의 경우, Gatys 등등의 논문 "Gatys, L.A., Ecker, A.S., Bethge, M.: A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576 (2015)
"과 유사한 결과를 얻지만 3배 더 빠르다.
  - 초해상도의 경우, 지각 손실 함수로 훈련된 우리의 방법은 픽셀당 손실함수로 훈련된 방법과 비교하여 질 높은(fine) 세부사항을 더 잘 재구성(reconstruct)할 수 있다.

# 2. Related Work

# 2.1. Feed-forward image transformation
- 최근에, 다양한 feedforward 이미지 변환 작업은 해결되었다. 픽셀당 손실함수로 깊은 컨볼루젼 신경망을 훈련시킴으로써
- 의미론적 분할 방법(semantic segmentation method)은 입력 이미지에 대한 완전한 컨볼루젼 방식(fully convolutional manner)으로 네트워크를 작동(run)시키고, 픽셀당 분류 손실함수(per-pixel classification loss)로 훈련함으로써, 밀도 있는 장면 레이블(dense scene label)을 생성한다.
- 논문 "Zheng, S., Jayasumana, S., Romera-Paredes, B., Vineet, V., Su, Z., Du, D., Huang, C., Torr, P.H.: Conditional random fields as recurrent neural networks. In: Proceedings of the IEEE International Conference on Computer Vision. (2015) 1529–1537"은 픽셀당 손실함수(per-pixel loss)를 뛰어넘었다. 나머지 네트워크와 공동으로 훈련된 반복되는(recurrent) 레이어로써 CRF inference를 구성함으로써.
- 변환 네트워크(transformation network)의 구조(architecture)는 논문 "Long, J., Shelhamer, E., Darrell, T.: Fully convolutional networks for semantic segmentation. CVPR (2015)", "Noh, H., Hong, S., Han, B.: Learning deconvolution network for semantic segmentation. arXiv preprint arXiv:1505.04366 (2015)"에 영감을 얻었다. 네트워크상의(in-network) 다운샘플링(downsampling)을 사용하여 feature map의 공간적인 크기(spatial extent)를 줄였다. 그 feature map은 마지막 출력 이미지를 생성하는 네트워크상의 업샘플링(upsampling)에 의한 것이다.
- 깊이(depth), 표면 정규 추정(surface normal estimation)의 최신의 방법은 그들이 픽셀당 회귀(regression) 또는 분류 손실함수를 사용하여 훈련된 feed-forward 컨볼루젼 네트워크를 사용하여 컬러 입력 이미지를 기하학적인 의미있는 출력 이미지로 변환하는 것과 비슷하다.
- 몇몇의 방법은 픽셀당 손실함수를 넘어서 이동한다. 이미지 기울기(image gradient)에 패널티를 부여하거나(penalize) CRF 손실 레이어를 사용하여 출력 이미지의 local 일관성(consistency)를 강화함(enforce)으로써.
- feed-forward 모델은 픽셀당 손실함수를 사용하여 훈련되었다. 회색조(grayscale) 이미지를 컬러 이미지로 변환하기 위해.


# 2.2. Perceptual optimization(지각 최적화)
- 최근의 여러 논문은 최적화(optimization)를 사용하여 이미지를 생성했다. 객체는 지각적이고, 컨볼루젼 네트워크에서 추출된 고수준의 feature에 의존한다.
- 이미지는 훈련된 네트워크에서 인코딩된 기능을 이해하기 위해서 클래스 예측 점수(class prediction score) 또는 개별 특징(individual feature)을 최대화하여 생성될 수 있다.
- 유사한 최적화 기술은 신뢰도(confidence)가 높은 가짜(fooling) 이미지 또한 생성할 수 있다.
- Mahendran, Vedaldi는 다른 네트워크 레이어를 포함(retain)하는 이미지 정보를 이해하기 위해서 특징 재구성 손실(feature reconstruction loss)을 최소화함으로써 컨볼루젼 네트워크에서부터 feature를 반전(invert)시켰다. 유사한 방법은 이전에 local binary descriptor, HOG feature를 반전시키는데 사용됬었다.
- Dosovitskiy, Brox의 작업은 특히 우리의 작업과 관련이 있다. 그들은 feed-forward 신경망을 convolutional feature를 반전시키기 위해 훈련시켰기 때문이다. 최적화 문제의 해결책을 신속하게 근사시킴으로써.
- 그러나, 그들의 feed-forward 네트워크는 픽셀당 재구성 손실함수로 훈련된 반면에, 우리의 네트워크는 feature 재구성 손실 함수를 직접적으로 최적화한다.

# 2.3. Style Transfer(스타일 전송)
- Gatys 등등은 한 이미지의 content를 다른 이미지의 style과 결합함으로써 예술적 스타일 전송(artistic style transfer)를 실행했다. 특징 재구성 손실함수(feature reconstruction loss)와 스타일 재구성 손실함수(style reconstruction loss)을 공동으로 최소화함으로써. 여기서 스타일 재구성 손실함수는 사전학습된 컨볼루젼 네트워크로부터 추출된 feature에 기반한다. ; 유사한 방법이 이전의 텍스쳐 합성(texture synthesis)에 사용되었다.
- 이들의 방법은 고품질의 결과를 생성한다. 하지만, 그들의 방법은 계산적으로 비싸다(computationally expensive). 왜냐하면, 그 최적화 문제의 각 단계는 사전훈련된 네트워크를 통해서 전진(forward), 후진(backward) 통과가 필요하기 때문이다.
- 이러한 계산적인 부담(computational burden)을 극복하기 위해서, 우리는 feed-forward 네트워크를 훈련시킨다. 그들의 최적화 문제를 위한 해결책에 거의 근사치를 내기 위해서.

# 2.4. Image super-resolution(이미지 초고해상도)
- 이미지 초고해상도는 다양한 기술이 개발된 고전적인 문제(classic problem)이다.
- Yang 등등은 컨볼루젼 신경망이 널리 채택되기 전에 널리 사용되는 기술에 대한 철저한 평가(exhaustive evaluation)를 제공한다.
- 그들은 초해상도 기술(super-resolution technique)을 예측기반 방법(prediction-based method), 엣지기반 방법(edgebased method), 통계적 방법(statistical method), 패치기반 방법(patch-based method), 희소사전 방법(sparse dictionary method)으로 무리지었다. (bilinear, bicubic, Lanczos 알고리즘)
- 최근에, 픽셀당 유클리디안 손실함수(per pixel Euclidean loss)로 훈련된 3-layer 컨볼루젼 신경망을 사용하여 단일 이미지 초고해상도에서 우수한 성능을 달성했다.

# 3. Method
- 그림 2에서 보이듯이, 우리의 시스템은 2개의 요소(component)와 일치한다. : 이미지 변환 네트워크(image transformation network) $f_W$, 손실 네트워크(loss network) $\phi$. 여러 손실 함수 $l_1, ..., l_k$를 정의하는 데 사용된다.
- <img src="https://blog.kakaocdn.net/dn/cP3KFR/btqEcGCMxki/KkduOGoEPBJDkkGHuEpPq1/img.png" width=800>

  - 그림 2.
  - 시스템 개요(system overview)
  - 우리는 입력 이미지를 출력 이미지로 변환하기 위해서 이미지 변환 네트워크를 훈련시킨다.
  - 이미지 분류를 위해 사전학습시킨 loss network를 사용하여 지각 손실 함수(perceptual loss function)를 정의한다. 그 지각 손실 함수는 이미지간의 content, style의 지각적인 차이를 측정한다.
  - loss network는 훈련 과정동안 고정된 상태로 유지된다.
- 이미지 변환 네트워크(image transformation network)는 가중치 W에 의해 매개변수화된 깊은 잔여 컨볼루젼 신경망(deep residual convolutional neural network)이다.; 이미지 변환 네트워크는 입력 이미지 $x$를 출력 이미지 $\hat{y}$로 변환시킨다. mapping $\hat{y} = f_W(x)$를 통해서.
- 각각의 손실 함수는 $l_{i}(\hat{y}, y_{i})$ 스칼라값을 계산한다. 출력 이미지 $\hat{y}$과 타겟 이미지 $y_{i}$ 간의 차이를 측정함으로써.
- 이미지 변환 네트워크는 손실 함수의 가중치 조합을 최소화하기 위해 확률적 경사 하강법을 사용하여 훈련된다. : $W^{∗} = arg min_{W}\mathbf{E}_{x, \{y_{i}\}} \left[ \sum_{i=1}\lambda_{i}l_{i}(f_{W}(x), y_{i}) \right]$ $\qquad$(1)
- 픽셀당 손실함수의 단점을 해결하고 이미지들간의 지각적, 의미론적인 차이를 우리의 손실 함수가 더 잘 측정할 수 있도록 하기 위해서, 우리는 최적화를 통해 이미지를 생성하는 최근의 작업에서 영감을 얻는다.
- 이러한 방법에서의 핵심 통찰력(key insight)은 이미지 분류를 위해 사전 훈련된 컨볼루젼 신경망이 이미 학습되었다는 것이다. 손실 함수에서 측정하려는 지각적, 의미론적인 정보를 인코딩하기 위해서.
- 그러므로, 우리는 손실 함수를 정의하기 위해서, 고정된 손실 네트워크로써, 이미지 분류를 위한 사전 훈련된 네트워크 $\phi$를 사용한다.
- 우리의 심층 컨볼루젼 변환 네트워크(deep convolutional transformation network)는 심층 컨볼루젼 네트워크인 손실 함수를 사용하여 훈련시킨다.
- 이미지간의 content, style 사이의 차이를 측정하는 특징 재구성 손실(feature reconstruction loss) $l_{feat}^{\phi}$과 스타일 재구성 손실(style reconstruction loss) $l_{style}^{\phi}$을 정의하는 데 손실 네트워크(loss network) $\phi$가 사용된다.
- 각각의 입력 이미지 $x$에 대해서, 우리는 content target $y_c$와 style target $y_s$가 있다.
- style transfer의 경우, content target $y_c$는 입력 이미지 $x$이고, 
- 출력 이미지 $\hat{y}$는 $x = y_c$의 content와 $y_s$의 style을 를 결합해야만 한다.; 우리는 style target 하나당 1개의 네트워크를 훈련시킨다.
- 단일 이미지 초고해상도의 경우, 입력 이미지 $x$는 저해상도 입력이다. content target $y_c$는 지상 실측 고해상도 이미지이다. 스타일 재구성 손실함수는 사용되지 않는다. ; 우리는 초고해상도 요소(factor) 하나당 1개의 네트워크를 훈련시킨다.


# 3.1. Image Transformation Networks(이미지 변환 네트워크)
- 우리의 이미지 변환 네트워크는 대략 Radford 등등에 의한 구조 기준(architectural guideline)을 따른다.
- 우리는 어떠한 풀링 레이어를 사용하지 X. 대신, 네트워크상의 다운샘플링, 업샘플링을 위해 스트라이드, 아주 약간의 스트라이드된 컨볼루젼을 사용한다.
- 우리의 네트워크 본체는 논문 "Gross, S., Wilber, M.: Training and investigating residual nets. http://torch.ch/blog/2016/02/04/resnets.html (2016)"의 구조를 사용하는 5개의 residual block으로 구성된다.
- 모든 non-residual convolutional layer에는 공간적 배치 정규화(spatial batch normalization)와 ReLU 비선형성(nonlinearity)이 따라온다. 출력 레이어를 제외하고 스케일링된 tanh 함수를 사용하여 출력 이미지가 [0, 255] 범위의 픽셀값을 갖도록 만든다.
- 9x9 커널을 사용하는 1번째, 마지막 레이어를 제외하고, 모든 컨볼루젼 레이어는 3x3 커널을 사용한다.
- 모든 우리의 네트워크의 정확한 구조는 보충 자료에서 찾을 수 있다.

# 3.1.1. Inputs and Outputs
- 스타일 전송의 경우, 입력 및 출력은 모두 3x256x256 shape의 컬러 이미지이다.
- 업샘플링 계수(factor)가 $f$인 초고해상도의 경우, 출력은 3x288x288 shape의 고해상도 이미지 패치이다. 입력은 3x288/$f$x288/$f$ shape의 저해상도 패치이다.
- 이미지 변환 네트워크는 fully convolutional이기 때문에, 테스트할 때, 이미지 변환 네트워크는 모든 해상도의 이미지에 적용할 수 있다.

# 3.1.2. Downsampling and Upsampling
- 업샘플링 계수(upsampling factor)가 $f$인 초해상도의 경우, 우리는 여러개의 잔차 블록(residual block)을 사용했다. stride 1/2의 $log_{2}f$ 컨볼루젼 레이어에 의한.
- 이것은 네트워크로 전달하기 전에 저해상도(low-resolution) 입력을 업샘플링하기 위해 bicubic 보간법(interpolation)을 사용하는 것과는 다르다.
- 고정된 업샘플링 함수에 의존하는 것 대신에, 아주 약간 스트라이드된 컨볼루젼은 나머지 네트워크와 함께 공동으로 학습할 수 있는 업샘플링 함수를 허락한다.
- 스타일 전송을 위해, 우리의 네트워크는 2개의 stride-2 컨볼루젼을 사용한다. 몇 개의 잔여 블록(residual block)에 의한 입력을 다운샘플링하기 위해서. 그리고, stride 1/2인 2개의 컨볼루젼 레이어를 업샘플링하기 위해서.
- 입력과 출력의 크기는 같지만, 다운샘플링을 한 다음에 업샘플링을 하는 네트워크에는 몇 가지 이점들이 있다.
- 이점 1. 계산(computational)
  - 순진한 구현(naive implementation)에서, $C × H × W$ 크기의 입력에서 $C$ 필터를 사용하는 3x3 컨볼루젼에는 $9HWC^2$ multiply-add가 필요하다. 이는 #$DC × H/D × W/D$ 크기의 입력에서 $DC$ 필터를 사용하는 3x3 컨볼루젼과 동일한 비용(cost)이다.
  - 따라서 다운샘플링 후에, 우리는 동일한 계산 비용(computational cost)으로 더 큰 네트워크를 사용할 수 있다.
- 이점 2. 효과적인 수용 필터의 크기(effective receptive field size)
  - 고품질 스타일 전송은 이미지의 많은 부분을 일관된(coherent) 방식으로 변경할 필요가 있다.; 그러므로 출력의 각 픽셀이 입력에 큰 효과적인 수용 필드를 갖는 것이 유리하다.
  - 다운샘플링이 없을 경우, 각각의 추가적인 3x3 컨볼루젼 레이어는 효과적인 수용 필드의 크기를 2씩 증가시킨다.
  - 계수 $D$로 다운샘플링이 일어난 후에, 각각의 3x3 컨볼루젼은 효과적인 수용 필드 크기를 2D만큼 증가시킨다. 동일한 수의 레이어로 더 큰 효과적인 수용 필드를 제공함으로써.

# 3.2. Perceptual Loss Functions
- 